In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
DATA_DIR = '../../data'
VIDEO_CAPS = DATA_DIR + '/msvd_video_caps.pkl'
VOCABULARY = DATA_DIR + '/vocab.pkl'

YOUTUBE_MAPPING = DATA_DIR + '/youtube_video_to_id_mapping.txt'
TRAIN_CAPS_PATH = DATA_DIR + '/sents_train_lc_nopunc.txt'
TEST_CAPS_PATH = DATA_DIR + '/sents_test_lc_nopunc.txt'
VAL_CAPS_PATH = DATA_DIR + '/sents_val_lc_nopunc.txt'

In [3]:
class Vocabulary():
    def __init__(self):
        pass

In [4]:
# load vocabulary used by the model
vocab = pickle.load(open(VOCABULARY, 'rb'))

In [5]:
mapping = pd.read_csv(YOUTUBE_MAPPING, sep=' ',header=None, names=["yt_id", "vid_id"])
mapping = mapping.set_index('vid_id')
mapping_dict = mapping.to_dict().get('yt_id')

In [11]:
def word2id(x):
    if vocab.word2idx.get(x) is None:
        return vocab.word2idx.get('<unk>')
    else: return vocab.word2idx.get(x)

def sent2ids(x):
    tmp = [vocab.word2idx.get('<start>')] + [word2id(y) for y in x.split(' ')] + [vocab.word2idx.get('<end>')]
    return np.array(tmp)

# load files
train_caps = pd.read_csv(TRAIN_CAPS_PATH, sep='\t',header=None, names=["vid_id", "cap"])
test_caps = pd.read_csv(TEST_CAPS_PATH, sep='\t',header=None, names=["vid_id", "cap"])
val_caps = pd.read_csv(VAL_CAPS_PATH, sep='\t',header=None, names=["vid_id", "cap"])

# map video id to youtube id
train_caps['vid_id'] = train_caps['vid_id'].transform(lambda x: mapping_dict.get(x))
test_caps['vid_id'] = test_caps['vid_id'].transform(lambda x: mapping_dict.get(x))
val_caps['vid_id'] = val_caps['vid_id'].transform(lambda x: mapping_dict.get(x))

# transform captions
train_caps['cap'] = train_caps['cap'].transform(sent2ids)
test_caps['cap'] = test_caps['cap'].transform(sent2ids)
val_caps['cap'] = val_caps['cap'].transform(sent2ids)

# select 5 sentences
train_caps5 = train_caps.groupby('vid_id').head(20)
test_caps5 = test_caps.groupby('vid_id').head(20)
val_caps5 = val_caps.groupby('vid_id').head(20)

In [12]:
np.save(DATA_DIR + '/val_new20.npy', val_caps5.values)
np.save(DATA_DIR + '/test_new20.npy', test_caps5.values)
np.save(DATA_DIR + '/train_new20.npy', train_caps5.values)

In [24]:
df1 = train_caps.sample(frac=1).groupby('vid_id').head(5)
df1[df1['vid_id'] == '7bcV-TL9mho_0_7']
#train_caps[train_caps['vid_id'] == '-4wsuPCjDBc_5_15']

,vid_id,cap
12828,7bcV-TL9mho_0_7,"[1, 1541, 7262, 7578, 9903, 6825, 2]"
12808,7bcV-TL9mho_0_7,"[1, 9903, 6825, 4537, 1788, 4542, 10071, 5552,..."
12813,7bcV-TL9mho_0_7,"[1, 9903, 10144, 5552, 7262, 1065, 7986, 9903,..."
12807,7bcV-TL9mho_0_7,"[1, 9903, 10144, 5552, 7262, 10100, 9048, 6244..."
12811,7bcV-TL9mho_0_7,"[1, 7262, 10100, 5111, 4145, 9903, 6825, 2]"
